In [1]:
import os
import sys
sys.path.append(os.path.abspath('..'))

import torch
from torch.utils.data import DataLoader
from transformer_lens import HookedTransformer
from transformer_lens import utils as tutils
from transformer_lens.evals import make_pile_data_loader, evaluate_on_dataset

from functools import partial
from datasets import load_dataset
from tqdm import tqdm

from sae_lens import SparseAutoencoder
from sae_lens.toolkit.pretrained_saes import get_gpt2_res_jb_saes
from sae_lens import SparseAutoencoder, ActivationsStore

from steering.eval_utils import evaluate_completions
from steering.utils import text_to_sae_feats, top_activations, normalise_decoder, get_activation_steering
from steering.patch import generate, get_scores_and_losses, patch_resid, get_loss, scores_2d

from sae_vis.data_config_classes import SaeVisConfig
from sae_vis.data_storing_fns import SaeVisData

import plotly.express as px
import plotly.graph_objects as go

torch.set_grad_enabled(False)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = HookedTransformer.from_pretrained("gemma-2b", device=device)

/home/slava/safety/steering/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Gemma's activation function should be approximate GeLU and not exact GeLU.
Changing the activation function to `gelu_pytorch_tanh`.if you want to use the legacy `gelu`, edit the `model.config` to set `hidden_activation=gelu`   instead of `hidden_act`. See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model gemma-2b into HookedTransformer


In [3]:
hp6 = "blocks.6.hook_resid_post"

sae6 = SparseAutoencoder.from_pretrained("gemma-2b-res-jb", hp6)
normalise_decoder(sae6, scale_input=False)
sae6 = sae6.to(device)

In [4]:
anger_dec = sae6.W_dec[1062]
# blog_dec = sae6.W_dec[14586]
wedding_dec = sae6.W_dec[8406]

scales = [0, 10, 20, 30, 40, 50, 60, 70, 80]
scales = [0, 40, 70]
prompt = "I think"


In [5]:
anger_scores, blog_scores, losses = scores_2d(
    model,
    hp6,
    steering_vectors=[anger_dec, wedding_dec],
    prompt=prompt,
    criterions=["Text is angry/upset/enraged/distressed or is about anger/hate etc.",
                "Text mentions wedding/marriage/engagement."],
    scales=scales,
    n_samples=5,
)


loading dataset: NeelNanda/c4-code-20k
dataset loaded


100%|██████████| 9/9 [10:10<00:00, 67.89s/it]


loss grid
tensor([[2.0377, 2.0439, 2.0647, 2.1067, 2.1813, 2.3021, 2.4917, 2.7871, 3.2093],
        [2.0442, 2.0524, 2.0759, 2.1223, 2.2035, 2.3351, 2.5427, 2.8627, 3.3087],
        [2.0721, 2.0834, 2.1122, 2.1664, 2.2599, 2.4103, 2.6455, 2.9971, 3.4707],
        [2.1372, 2.1539, 2.1912, 2.2585, 2.3720, 2.5516, 2.8239, 3.2130, 3.7106],
        [2.2563, 2.2818, 2.3324, 2.4188, 2.5604, 2.7798, 3.0980, 3.5237, 4.0354],
        [2.4416, 2.4795, 2.5478, 2.6603, 2.8390, 3.1063, 3.4709, 3.9234, 4.4365],
        [2.7047, 2.7606, 2.8544, 3.0003, 3.2215, 3.5353, 3.9347, 4.3978, 4.8982],
        [3.0630, 3.1434, 3.2688, 3.4534, 3.7155, 4.0616, 4.4748, 4.9296, 5.4044],
        [3.5303, 3.6408, 3.8013, 4.0216, 4.3105, 4.6658, 5.0701, 5.5011, 5.9413]],
       device='cuda:0')
evaluating (0, 0)


100%|██████████| 50/50 [00:54<00:00,  1.08s/it]


['I think the answer is d)\nWe can see from the answer provided that:\nThe probability that the', "I think that's the best price ever for a used one on the 4G as there's", 'I think the only two ways you could buy a 2nd car are either a lease or a loaner', "I think the title sums up the issue. I'm trying to import a new project and it can read", "I think its pretty much decided to be a boy. I'm 19 weeks. Went in this", "I think it'd be cute if they made that into an episode (I love Garmains and Az", 'I think it’s common that a lot of people feel like they have a very short amount of time to', 'I think I’m in the minority here, but I don’t like that “I’m having', "I think the thing is we can't really be too certain what happened for sure. It's hard", 'I think it’s safe to say that it’s been a long time since a <em>Fast and', "I think the key to your answer is to recognize that the point of a model is that it doesn't", 'I think there is a lack of detail regarding all the things you have t

...

['I think its not about with the post but it i write because....\n\nThe reason I get really about is', 'I think I want for a more, because when people, usually men, at a and or. This the', 'I think, at this point,! in. at! |. about for because! at... ... when', "I think I over. I' now i that I am mad because and a little to angry or at I", 'I think for today today,, it’ged,\n\nI’’’’ I’s  that', "I think what you's on a about you' about about that that you''s blog about how", 'I think about a lot about a lot about how how and most of the time. and at the beginning if', 'I think the article below is about about a pretty post. that you don at and feel all about because and', "I think that people don'. at today and because. that I' at a that about. and and", 'I think a lot of people about about that post on a blog on   of about for this. But', "I think about about that and it'gedged....!\n\nI'gedgedgedblog.", 'I think I’s pretty quiet because about a week ago about  with a post about that 

100%|██████████| 50/50 [00:53<00:00,  1.07s/it]


["I think many and how are are you that you'ged and because that that when you get that anger is", 'I think it took years and about a lot and a lot of about because on my blog post.\n\nI', 'I think about a lot.. about is a, about I is - - - - and you will at', 'I think I because.\nI thought about this before that....\nBut still that I write more...', 'I think I read the the post of with the name that is, but about at this that about a comment', 'I think you really about about about because for of an in about that and mad, or. What angry about', 'I think these more (as well as about of!) about a few weeks ago and and now i justs', 'I think that I should so. now here about a little this ... or, blog about so blog ... ...', "I think this is wrong because that's the I that :\n\nAngry, that', mad that ...", 'I think it was at a friend, I just about at, but in a mood or or as angry,', "I think I might and with that is a about post. that and when over it is I. i'", 'I think I’s about as mad a

100%|██████████| 50/50 [00:53<00:00,  1.07s/it]


['I think that I about with this blog. about post about -\n\nabout about -\n\n... post a...', "I think about and about about everyday and write angry, as if… and () I's so,", 'I think I should about that...\n\n<blockquote><b>About</b><b>:</b> <b> -  </b> ', 'I think that more often than than about over… on and or, about and () a lots... about', 'I think I have had about enough about about a blog post or... for or or for ever on post-', 'I think it was on about about for not posts in at that stage that about at at was I about about', 'I think a lot about because of the, when I see about. A is at angry and because I about', 'I think,. because I blogged about after I, about.. or I and that.... about', 'I think I’ is with I’s a lot because that people… when mad with their wife about and', 'I think that about when I that about when about..\n\n\nBlogspotpost\n\nAngry about about I mad', 'I think this about as it.\n\nWhen I first about a and about blog posts.\n\n about post!', 'I think about 

In [28]:
px.imshow(anger_scores, x=scales, y=scales, title="Anger scores",labels={'x': 'blog', 'y': 'anger'}, color_continuous_scale="RdBu", color_continuous_midpoint=0).show()

In [30]:
px.imshow(blog_scores, x=scales, y=scales, title="Blog post scores",labels={'x': 'blog', 'y': 'anger'}, color_continuous_scale="RdBu", color_continuous_midpoint=0).show()

In [31]:
px.imshow(losses, x=scales, y=scales, title="Losses",labels={'x': 'blog', 'y': 'anger'}, color_continuous_scale="RdBu", color_continuous_midpoint=0).show()

In [32]:
px.imshow(anger_scores+blog_scores, x=scales, y=scales, title="sum of scores",labels={'x': 'blog', 'y': 'anger'}, color_continuous_scale="RdBu", color_continuous_midpoint=0).show()

In [9]:
# save_dir = "heatmaps/anger_blog"

### save
# os.makedirs(save_dir, exist_ok=True)
# torch.save(anger_scores, os.path.join(save_dir, "anger_scores.pt"))
# torch.save(blog_scores, os.path.join(save_dir, "blog_scores.pt"))
# torch.save(losses, os.path.join(save_dir, "losses.pt"))

### load
# anger_scores = torch.load(os.path.join(save_dir, "anger_scores.pt"))
# blog_scores = torch.load(os.path.join(save_dir, "blog_scores.pt"))
# losses = torch.load(os.path.join(save_dir, "losses.pt"))